# **Line Intercept Vegetation Calculations**

**this notebook contains code for calculations of percent cover of various cover types across:**
1. The entire transect (to HTS)
2. the foredune portion of the transect (dune heel to dune toe)
3. the vegetated portion of the transect (to lowest veg)

## Import Packages

In [1]:
import pandas as pd

## Import Data

replace the file_path within the parentheses '' with the actual path to your excel file containing the raw data. 

Note: Your excel data must be formatted according to the following sheet to work with the rest of the code:

https://docs.google.com/spreadsheets/d/1tFZVjsOSf4BBCrjyVIJQeS-zN7qWJvLR3fg2mDO5wUA/edit?usp=sharing 

In [2]:
#file path
file_path = '/Users/mayabernstein/Documents/SeaGrantVeg/combined_data.xlsx' #file path


## Load Data From Each Sheet

In [3]:
positional_df = pd.read_excel(file_path, sheet_name="PositionalCharacteristics")
transects_df = pd.read_excel(file_path, sheet_name="Transects")
elevation_df = pd.read_excel(file_path, sheet_name="Elevation")
readme_df = pd.read_excel(file_path, sheet_name="ReadMe")

## Create a Dataframe

This will be where the calculated variables are stored

In [4]:
calculations_df = pd.DataFrame()

## Supplement Transect Data

This adds new columns to identify if the rows are:
1. vegetation 
2. within the foredune
3. within the vegetated portion of the tranesct

In [5]:
# Strip leading/trailing spaces to ensure clean matching
transects_df['type'] = transects_df['type'].str.strip()
readme_df['name'] = readme_df['name'].str.strip()

#amend the transects column to have more specific data so there are no duplicate values
transects_df["transect"] = transects_df["sitename"] + "_" + transects_df["date"].astype(str) + "_" + transects_df["transect"]
positional_df["transect"] = positional_df["sitename"] + "_" + positional_df["date"].astype(str) + "_" + positional_df["transect"]

# Create a mapping from 'name' to 'native' from readme_df
name_to_native = readme_df.set_index('name')['native']

# Add 'native' column to transects_df based on 'type' matching 'name'
transects_df['native'] = transects_df['type'].map(name_to_native)

# Create a mapping from 'name' to 'codetype' in readme_df
type_to_codetype = readme_df.set_index('name')['codetype']
# Map the 'codetype' values to transects_df based on 'type'
transects_df['codetype'] = transects_df['type'].map(type_to_codetype)
# Fill missing 'codetype' values with "Dead Terrestrial Plant" if 'type' contains "-D"
transects_df.loc[transects_df['type'].str.contains('-D', na=False), 'codetype'] = "Dead Terrestrial Plant"


# Create a mapping for positional information (toe_sea, toe_in, lowest_veg)
positional_mapping = positional_df.set_index('transect')[['toe_sea', 'toe_in', 'lowest_veg']]

# Map the positional values to transects_df
transects_df = transects_df.join(positional_mapping, on='transect')

# Calculate if the row starts or ends within the dune
start_within_dune = (transects_df['start'] <= transects_df['toe_sea']) & (transects_df['start'] >= transects_df['toe_in'])
end_within_dune = (transects_df['end'] >= transects_df['toe_in']) & (transects_df['end'] <= transects_df['toe_sea'])

# Create the final 'dune' column directly
transects_df['dune'] = start_within_dune | end_within_dune


# Identify if the row is within the vegetated portion of the dune
transects_df['veg'] = transects_df['start'] <= transects_df['lowest_veg']

#transects_df.drop(columns=["sitename", "date"], inplace=True)

## Calculate and build in variables

In [6]:
#transect letter
calculations_df["transect"] = positional_df["transect"]
#transect name and site name
calculations_df["sitename"] = positional_df ["sitename"]
calculations_df["date"] = positional_df["date"]
# transect length
calculations_df["tran_length"] = (positional_df["HTS"] - positional_df["eastend"]).abs()
 # dune length
calculations_df["dune_length"] = positional_df["toe_sea"] - positional_df["toe_in"].abs()
# vegeted length
calculations_df["veg_length"] = (positional_df["lowest_veg"] - positional_df["eastend"]).abs()

# -------all cover types------

# EVERYTHING OVER ENTIRE TRANSECT
calculations_df = calculations_df.merge(
    transects_df.groupby("transect")["cor_length"].sum().reset_index(),
    on="transect", how="left"
)
calculations_df["pctcov_all_whole"] = calculations_df["cor_length"] / calculations_df["tran_length"]
calculations_df.drop(columns=["cor_length"], inplace=True)

# EVERYTHING OVER DUNE PORTION OF TRANSECT
dune_df = transects_df[transects_df["dune"] == True] \
    .groupby("transect", as_index=False)["cor_length"].sum()

calculations_df = calculations_df.merge(dune_df, on="transect", how="left")
calculations_df["cor_length"] = calculations_df["cor_length"].fillna(0)  # Fill missing values with 0
calculations_df["pctcov_all_dune"] = calculations_df["cor_length"] / calculations_df["dune_length"]
calculations_df.drop(columns=["cor_length"], inplace=True)

# EVERYTHING OVER VEGETATED PORTION OF TRANSECT
veg_df = transects_df[transects_df["veg"] == True] \
    .groupby("transect", as_index=False)["cor_length"].sum()

calculations_df = calculations_df.merge(veg_df, on="transect", how="left")
calculations_df["cor_length"] = calculations_df["cor_length"].fillna(0)  # Fill missing values with 0
calculations_df["pctcov_all_veg"] = calculations_df["cor_length"] / calculations_df["veg_length"]
calculations_df.drop(columns=["cor_length"], inplace=True)


#_______________TRANSECT WIDE_____

# Convert codetype to string and replace NaN values with "Unknown"
transects_df["codetype"] = transects_df["codetype"].astype(str).fillna("Unknown")
# Get unique codetype values
unique_codetypes = transects_df["codetype"].unique()

# Loop through each codetype and calculate percent cover
for codetype in unique_codetypes:
    codetype_clean = f"pctcov_{codetype.replace(' ', '')}_transect"  # Remove spaces for column name
    # filter transect_df to only include rows where codetype matches current iteration
    temp_df = transects_df[transects_df["codetype"] == codetype] \
        .groupby("transect", as_index=False)["cor_length"].sum() # sums the cor_length of each veg category for each transect

    # Merge with calculations_df
    calculations_df = calculations_df.merge(temp_df, on="transect", how="left")
    # Fill NaN values in 'cor_length' with 0 before calculating percent cover
    calculations_df["cor_length"] = calculations_df["cor_length"].fillna(0)
    # Calculate percent cover for the codetype
    calculations_df[codetype_clean] = calculations_df["cor_length"] / calculations_df["tran_length"]
    # Drop 'cor_length' column
    calculations_df.drop(columns=["cor_length"], inplace=True)

# native/nonnative plants

# Define column name for native terrestrial plant percent cover
native_terrestrial_col = "pctcov_TerrestrialPlantNative_transect"
# Filter for only native terrestrial plants
native_terrestrial_df = transects_df[(transects_df["codetype"] == "Terrestrial Plant") & (transects_df["native"] == 1.0)] \
    .groupby("transect", as_index=False)["cor_length"].sum()
# Merge with calculations_df
calculations_df = calculations_df.merge(native_terrestrial_df, on="transect", how="left")
# Fill NaN values in 'cor_length' with 0 before calculating percent cover
calculations_df["cor_length"] = calculations_df["cor_length"].fillna(0)
# Calculate percent cover for native terrestrial plants
calculations_df[native_terrestrial_col] = calculations_df["cor_length"] / calculations_df["tran_length"]
# Drop 'cor_length' column
calculations_df.drop(columns=["cor_length"], inplace=True)

# Define column name for native terrestrial plant percent cover
nonnative_terrestrial_col = "pctcov_TerrestrialPlantNonnative_transect"
# Filter for only nonnative terrestrial plants
nonnative_terrestrial_df = transects_df[(transects_df["codetype"] == "Terrestrial Plant") & (transects_df["native"] == 0.0)] \
    .groupby("transect", as_index=False)["cor_length"].sum()
# Merge with calculations_df
calculations_df = calculations_df.merge(nonnative_terrestrial_df, on="transect", how="left")
# Fill NaN values in 'cor_length' with 0 before calculating percent cover
calculations_df["cor_length"] = calculations_df["cor_length"].fillna(0)
# Calculate percent cover for native terrestrial plants
calculations_df[nonnative_terrestrial_col] = calculations_df["cor_length"] / calculations_df["tran_length"]
# Drop 'cor_length' column
calculations_df.drop(columns=["cor_length"], inplace=True)



#--------DUNE----------

# Loop to calculate percent cover for each codetype over the dune portion of the transect
for codetype in unique_codetypes:
    codetype_clean_dune = f"pctcov_{codetype.replace(' ', '')}_dune"  # Clean column name for dune
    # filter transect_df to only include rows where codetype matches current iteration
    temp_df_dune = transects_df[(transects_df["codetype"] == codetype) & (transects_df["dune"] == True)] \
        .groupby("transect", as_index=False)["cor_length"].sum() # sums the cor_length of each veg category for each transect

    # Merge with calculations_df
    calculations_df = calculations_df.merge(temp_df_dune, on="transect", how="left")
    # Fill NaN values in 'cor_length' with 0 before calculating percent cover
    calculations_df["cor_length"] = calculations_df["cor_length"].fillna(0)
    # Calculate percent cover for the dune portion
    calculations_df[codetype_clean_dune] = calculations_df["cor_length"] / calculations_df["dune_length"]
    # Drop 'cor_length' column
    calculations_df.drop(columns=["cor_length"], inplace=True)

# native/nonnative plants

# Define column name for native terrestrial plant percent cover
native_terrestrial_dune_col = "pctcov_TerrestrialPlantNative_dune"
# Filter for only native terrestrial plants in the dune
native_terrestrial_dune_df = transects_df[(transects_df["codetype"] == "Terrestrial Plant") & (transects_df["native"] == 1.0) & (transects_df["dune"]== True)] \
    .groupby("transect", as_index=False)["cor_length"].sum()
# Merge with calculations_df
calculations_df = calculations_df.merge(native_terrestrial_dune_df, on="transect", how="left")
# Fill NaN values in 'cor_length' with 0 before calculating percent cover
calculations_df["cor_length"] = calculations_df["cor_length"].fillna(0)
# Calculate percent cover for native terrestrial plants
calculations_df[native_terrestrial_dune_col] = calculations_df["cor_length"] / calculations_df["dune_length"]
# Drop 'cor_length' column
calculations_df.drop(columns=["cor_length"], inplace=True)

# Define column name for native terrestrial plant percent cover
nonnative_terrestrial_dune_col = "pctcov_TerrestrialPlantNonnative_dune"
# Filter for only nonnative terrestrial plants
nonnative_terrestrial_dune_df = transects_df[(transects_df["codetype"] == "Terrestrial Plant") & (transects_df["native"] == 0.0) & (transects_df["dune"]== True)] \
    .groupby("transect", as_index=False)["cor_length"].sum()
# Merge with calculations_df
calculations_df = calculations_df.merge(nonnative_terrestrial_dune_df, on="transect", how="left")
# Fill NaN values in 'cor_length' with 0 before calculating percent cover
calculations_df["cor_length"] = calculations_df["cor_length"].fillna(0)
# Calculate percent cover for native terrestrial plants
calculations_df[nonnative_terrestrial_dune_col] = calculations_df["cor_length"] / calculations_df["dune_length"]
# Drop 'cor_length' column
calculations_df.drop(columns=["cor_length"], inplace=True)

#------------VEGETATED PORTION-----------------

# Loop to calculate percent cover for each codetype over the dune portion of the transect
for codetype in unique_codetypes:
    codetype_clean_veg = f"pctcov_{codetype.replace(' ', '')}_veg"  # Clean column name for dune
    # filter transect_df to only include rows where codetype matches current iteration
    temp_df_veg = transects_df[(transects_df["codetype"] == codetype) & (transects_df["veg"] == True)] \
        .groupby("transect", as_index=False)["cor_length"].sum() # sums the cor_length of each veg category for each transect

    # Merge with calculations_df
    calculations_df = calculations_df.merge(temp_df_veg, on="transect", how="left")
    # Fill NaN values in 'cor_length' with 0 before calculating percent cover
    calculations_df["cor_length"] = calculations_df["cor_length"].fillna(0)
    # Calculate percent cover for the dune portion
    calculations_df[codetype_clean_veg] = calculations_df["cor_length"] / calculations_df["veg_length"]
    # Drop 'cor_length' column
    calculations_df.drop(columns=["cor_length"], inplace=True)

# native/nonnative plants

# Define column name for native terrestrial plant percent cover
native_terrestrial_veg_col = "pctcov_TerrestrialPlantNative_veg"
# Filter for only native terrestrial plants in the dune
native_terrestrial_veg_df = transects_df[(transects_df["codetype"] == "Terrestrial Plant") & (transects_df["native"] == 1.0) & (transects_df["veg"]== True)] \
    .groupby("transect", as_index=False)["cor_length"].sum()
# Merge with calculations_df
calculations_df = calculations_df.merge(native_terrestrial_veg_df, on="transect", how="left")
# Fill NaN values in 'cor_length' with 0 before calculating percent cover
calculations_df["cor_length"] = calculations_df["cor_length"].fillna(0)
# Calculate percent cover for native terrestrial plants
calculations_df[native_terrestrial_veg_col] = calculations_df["cor_length"] / calculations_df["veg_length"]
# Drop 'cor_length' column
calculations_df.drop(columns=["cor_length"], inplace=True)

# Define column name for native terrestrial plant percent cover
nonnative_terrestrial_veg_col = "pctcov_TerrestrialPlantNonnative_veg"
# Filter for only nonnative terrestrial plants
nonnative_terrestrial_veg_df = transects_df[(transects_df["codetype"] == "Terrestrial Plant") & (transects_df["native"] == 0.0) & (transects_df["veg"]== True)] \
    .groupby("transect", as_index=False)["cor_length"].sum()
# Merge with calculations_df
calculations_df = calculations_df.merge(nonnative_terrestrial_veg_df, on="transect", how="left")
# Fill NaN values in 'cor_length' with 0 before calculating percent cover
calculations_df["cor_length"] = calculations_df["cor_length"].fillna(0)
# Calculate percent cover for native terrestrial plants
calculations_df[nonnative_terrestrial_veg_col] = calculations_df["cor_length"] / calculations_df["veg_length"]
# Drop 'cor_length' column
calculations_df.drop(columns=["cor_length"], inplace=True)


#______SPECIES SPECIFIC_____________

# Get unique species names
unique_species = transects_df["type"].dropna().unique()

# Loop through each species
for species in unique_species:
    species_clean = f"pctcov_{species.replace(' ', '')}_whole"  # Remove spaces for column name
    
    # Percent cover over entire transect
    temp_df = transects_df[transects_df["type"] == species] \
        .groupby("transect", as_index=False)["cor_length"].sum()
    
    calculations_df = calculations_df.merge(temp_df, on="transect", how="left")
    calculations_df["cor_length"] = calculations_df["cor_length"].fillna(0)
    calculations_df[species_clean] = calculations_df["cor_length"] / calculations_df["tran_length"]
    calculations_df.drop(columns=["cor_length"], inplace=True)

    # Percent cover over dune portion
    species_clean_dune = f"pctcov_{species.replace(' ', '')}_dune"
    temp_df = transects_df[(transects_df["type"] == species) & (transects_df["dune"] == True)] \
        .groupby("transect", as_index=False)["cor_length"].sum()
    
    calculations_df = calculations_df.merge(temp_df, on="transect", how="left")
    calculations_df["cor_length"] = calculations_df["cor_length"].fillna(0)
    calculations_df[species_clean_dune] = calculations_df["cor_length"] / calculations_df["dune_length"]
    calculations_df.drop(columns=["cor_length"], inplace=True)

    # Percent cover over vegetated portion
    species_clean_veg = f"pctcov_{species.replace(' ', '')}_veg"
    temp_df = transects_df[(transects_df["type"] == species) & (transects_df["veg"] == True)] \
        .groupby("transect", as_index=False)["cor_length"].sum()
    
    calculations_df = calculations_df.merge(temp_df, on="transect", how="left")
    calculations_df["cor_length"] = calculations_df["cor_length"].fillna(0)
    calculations_df[species_clean_veg] = calculations_df["cor_length"] / calculations_df["veg_length"]
    calculations_df.drop(columns=["cor_length"], inplace=True)


calculations_df.to_csv("calculations_output.csv", index=False)


